In [30]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
from numba import njit, jit, vectorize
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
from scipy import stats

First get weather data and metadata for a desired location (latitude and logitude)

In [2]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove and comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

c:\ProgramData\anaconda3\envs\monte-carlo\lib\site-packages\pvlib\iotools\psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [3]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995 
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 62.08 # average
sd_Ea = 7.3858 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_lnR0 = 13.7223084 
sd_lnR0 = 2.47334772

# number of iterations
n = 20000

In [4]:
# notice symmetry of matrix
A = np.array([[1,   Ea_X,   Ea_lnR0],
              [Ea_X,    1,   X_lnR0],
              [Ea_lnR0, X_lnR0,   1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True) 

# originally attempting to get a standard distribution based each parameter's mean and standard deviation for n points
# now: creates random distribution with mean = 0 and std = 1 for n points
ea = np.random.normal(loc=0, scale=1, size=n)
x = np.random.normal(loc=0, scale=1, size=n)
lnR0 = np.random.normal(loc=0, scale=1, size=n)


# create a numPy array to use in operations later
# somewhat misleadingly named a matrix instead of an array
# transpose built in so I don't have to do it with another function
samples_matrix = np.array([ea, x, lnR0])

# I kind of hate this 
data = {
    'ea': ea,
    'x': x,
    'lnR0': lnR0
}
uncorrelated_df = pd.DataFrame(data) # not sure if I am actually using this for anything


In [5]:
# correlated stats pre-input to function using MonteCarloEaLnRoX
correlated_samples = np.matmul(A_decomp, samples_matrix)

#correlated_df = pd.DataFrame(correlated_samples.T, columns=['ea', 'x', 'lnR0']) # dont think this even gets used

sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')

In [18]:
'''
general form taken to update our plain distrubution values with means and stdevs to make them meaningful 
Ea_New = sd_Ea * ea.std() + mean_Ea
lnR0_New = sd_lnR0 * lnR0.std() + mean_lnR0
x_NEW = sd_X * x.std() + mean_X
# visualize
print(Ea_New)
print(lnR0_New)
print(x_NEW)
'''

'\nEa_New = sd_Ea * ea.std() + mean_Ea\nlnR0_New = sd_lnR0 * lnR0.std() + mean_lnR0\nx_NEW = sd_X * x.std() + mean_X\n# visualize\nprint(Ea_New)\nprint(lnR0_New)\nprint(x_NEW)\n'

In [6]:
temp = np.matrix(np.matmul(A_decomp, samples_matrix))
sd_mat = np.matrix([sd_Ea, sd_X, sd_lnR0]) 

sd_mat_transpose = np.transpose(sd_mat)
result = np.multiply(temp, sd_mat_transpose) + np.transpose(np.matrix([mean_Ea, mean_X, mean_lnR0]))

correlated_df = pd.DataFrame(np.transpose(result), columns=['ea', 'x', 'lnR0'])

In [7]:
# dummy check for mean and standard deviation
print(result[0].mean())
print(result[0].std())
print()
print(result[1].mean())
print(result[1].std())
print()
print(result[2].mean())
print(result[2].std())
print()

print("EA_X:", np.corrcoef(result[0], result[1]))
print("Ea_lnR0:", np.corrcoef(result[0], result[2]))
print("X_lnR0:", np.corrcoef(result[1], result[2]))

62.02797421810896
7.454579394249402

0.03571335533754146
0.09855420709608427

13.739260328478316
2.496352956274537

EA_X: [[1.         0.02600425]
 [0.02600425 1.        ]]
Ea_lnR0: [[ 1.         -0.99952095]
 [-0.99952095  1.        ]]
X_lnR0: [[ 1.         -0.03847621]
 [-0.03847621  1.        ]]


In [59]:
# Kempe's function ported from excel 
@njit
def forArrenius(poa_global, module_temp, ea, x, lnR0): # add type hinting
    degredation = np.zeros_like(ea) # matches the number of samples in ea

    # moved precalculations outside of loop, much faster this way
    ea1 = ea / 8.31446261815324E-03
    R0 = np.exp(lnR0)
    poa_global_scaled = poa_global / 1000

    for i in range(n):
        for j in range(8760):
            # very inefficient, an element-wise approach would be siginficantly faster
            degredation[i] += R0[i] * np.exp(-ea1[i] / (273.15 + module_temp[j])) * np.power(poa_global_scaled[j], x[i])
            

    return (degredation / 8760)



In [61]:
### function testing ###
# 10-13 seconds for 20K samples with numba
for_deg = forArrenius(poa_global=poa_irradiance['poa_global'].to_numpy(), module_temp=temp_mod.to_numpy(), ea=correlated_df['ea'].to_numpy(), x=correlated_df['x'].to_numpy(), lnR0=correlated_df['lnR0'].to_numpy())

In [67]:
output_df = pd.DataFrame(for_deg)


# Print the first 20 rows of the DataFrame
print(output_df.head(20))

inf_values = output_df.isin([np.inf]).any(axis=1)
num_rows_with_inf = inf_values.sum()
print(num_rows_with_inf / n * 100, "% of rows have inf")

# for troubleshooting examine index = 3 input data to see what could be causing undesired behavior in function
# some pen and paper work if you will



               0
0   6.417456e-05
1   2.012880e-04
2   1.295920e-06
3            inf
4            inf
5            inf
6   6.057658e-03
7   3.579358e-04
8            inf
9            inf
10  4.271132e-06
11           inf
12           inf
13  3.921846e-06
14  1.653505e-01
15           inf
16           inf
17  5.431943e-05
18  9.737049e-07
19  5.662003e-02
35.82 % of rows
